In [20]:
#!/usr/bin/env python
from openpyxl import load_workbook
import glob
import os,sys
from openeye.oechem import *
import traceback
from parse import parse
import cx_Oracle

class BiogenDb:
    def __init__(self):
        self.conn = cx_Oracle.connect('chem_user','Biogenidec123','10.2.129.34:1725/PRELNR')

    def getMolFromBio(self, batchId):
        cursor = None
        try:
            cursor = self.conn.cursor()
            cursor.execute("select lot_chemistry,lot_corporate_id, project, collection_name, collection_subset from crdatamart.compound_lots where name = :batchId",(batchId,))
            result = cursor.fetchone()
            if result is not None:
                molfile,lot_id, projectName, collection_name, collection_subset = result
                mol = OEGraphMol()
                ifs = oemolistream()
                ifs.SetFormat(OEFormat_SDF)
                ifs.openstring(molfile.read())
                OEReadMolecule(ifs,mol)
                mol.SetTitle(lot_id)
                if projectName is None:
                    projectName = ""
                if collection_name is None:
                    collection_name = ""
                if collection_subset is None:
                    collection_subset = ""
                OESetSDData(mol,"Project",projectName)
                OESetSDData(mol,"CollectionName",collection_name)
                OESetSDData(mol,"CollectionSubset",collection_subset)
                return mol

            else:
                return None
        except:
            traceback.print_exc()
            return None
        finally:
            if cursor is not None:
                cursor.close()

    def __del__(self):
        self.conn.close()
        

db = BiogenDb()
mol = db.getMolFromBio("BIO-0917904-01")
print OEMolToSmiles(mol),mol.GetTitle(),OEGetSDData(mol,"Project"),\
    OEGetSDData(mol,"CollectionName"),OEGetSDData(mol,"CollectionSubset")

COCCOc1cc(cc2c1cnn2c3cccc(n3)CO)c4cccc(n4)[C@H](C5CCC5)N BIO-0917904-01 IRAK4  


In [25]:
import csv,os
db = BiogenDb()

class Sample:
    def __init__(self, batchId,barcode,labwareType,amount,unit,concentration,concUnit,setfolderName):
        self.batchId = batchId
        self.barcode = barcode
        self.labwareType = labwareType
        self.amount = amount
        self.unit = unit
        self.concentration = concentration
        self.concUnit = concUnit
        self.setFolderName = setfolderName
        self.oemol = db.getMolFromBio(self.batchId)
        if self.oemol is not None:
            self.project = OEGetSDData(self.oemol,"Project")
            self.collection_name = OEGetSDData(self.oemol,"CollectionName")
            self.collection_subset = OEGetSDData(self.oemol,"CollectionSubset")
            OESetSDData(self.oemol,"batchId",batchId)
            OESetSDData(self.oemol,"barcode",barcode)
            OESetSDData(self.oemol,"labwareType",self.labwareType)
            OESetSDData(self.oemol,"amount",self.amount)
            OESetSDData(self.oemol,"unit",self.unit)
            OESetSDData(self.oemol,"concentration",self.concentration)
            OESetSDData(self.oemol,"concentration unit",self.concUnit)
            try:
                self.bionumber = self.batchId.rsplit("-",1)[0]
            except:
                self.bionumber = self.batchId
            OESetSDData(self.oemol,"BIO-NUMBER",self.bionumber)
        
        
        

directory = "/Users/jfeng1/Inventory"
file = os.path.join(directory,"BiogenInventory.csv")
ofs = oemolostream()
ofs.open(os.path.join(directory,"BiogenInventory.sdf"))

csvfile = open(file,"r")
csvreader = csv.reader(csvfile)
colNames = {}
for row in csvreader:
    if csvreader.line_num == 1:
        for id,cell in enumerate(row):
            colNames[cell] = id
    else:
        batch_id = row[colNames['Batch Id']]
        barcode = row[colNames['Labware Barcode']]
        labwareType = row[colNames['Labware Type']]
        amount = row[colNames['Available Amount']]
        unit = row[colNames['Available Amount Unit']]
        concentration = row[colNames['Concentration']]
        concUnit = row[colNames['Conc. Unit']]
        setFolderName = row[colNames['SetFolderName']]
        sample = Sample(batch_id,barcode,labwareType,amount,unit,concentration,concUnit,setFolderName)
        if sample.oemol is not None:
            OEWriteMolecule(ofs,sample.oemol)
ofs.close()

In [3]:
volumnColName = "Volume UL"
grossColName = "Gross Weight"
tareColName = "Tare Weight"
bionumberColName = "Corporate ID"
lotColName = "Lot Number"
barcodeName = "Barcode"


if __name__ == "__main__":
    if len(sys.argv)!=2:
        print "Usage:%s input.xls"%sys.argv[0]
        print "First row contains data tag names"
        sys.exit(0)

    OEThrow.SetLevel(OEErrorLevel_Error)
    xlsFile = sys.argv[1]

    liquidDb = {}
    solidDb = {}

    liquidBarcodeDb = {}
    solidBarcodeDb = {}

    wb = load_workbook(xlsFile)

    print "Loading done."
    solid_sheet = wb.get_sheet_by_name("Solid")
    colNames = []
    first_row = solid_sheet.rows[0]
    for c in first_row:
        colName = c.value
        colNames.append(colName.encode('utf-8').strip())

    try:
        bio_number_col_id = colNames.index(bionumberColName)
        grossId = colNames.index(grossColName)
        tareId = colNames.index(tareColName)
        lotId = colNames.index(lotColName)
        barcodeId = colNames.index(barcodeName)
    except:
        print "No desired columns"
        sys.exit(1)

    for row in solid_sheet.rows[1:]:
        try:
            bio_number = str(row[bio_number_col_id].value)
            lot_number = str(row[lotId].value)
            gross_weight = float(row[grossId].value)
            tare_weight = float(row[tareId].value)
            weight = 1000*(gross_weight-tare_weight)
            barcode = str(row[barcodeId].value)
            if weight <= 0:
                continue
            key = "%s-%s"%(bio_number,lot_number)
            if not solidDb.has_key(key):
                solidDb[key] = weight
                solidBarcodeDb[key] = []
                solidBarcodeDb[key].append(barcode)
            else:
                solidDb[key] = solidDb[bio_number]+weight
                solidBarcodeDb[key].append(barcode)
        except:
            traceback.print_exc()
            pass


    colNames = []
    liquid_sheet = wb.get_sheet_by_name("Liquid")
    first_row = liquid_sheet.rows[0]
    for c in first_row:
        colName = c.value
        colNames.append(colName.encode('utf-8').strip())

    bio_number_col_id = colNames.index(bionumberColName)
    volumColId = colNames.index(volumnColName)
    lotId = colNames.index(lotColName)
    barcodeId = colNames.index(barcodeName)
    for row in liquid_sheet.rows[1:]:
        try:
            bio_number = str(row[bio_number_col_id].value)
            lot_number = str(row[lotId].value)
            barcode = str(row[barcodeId].value)
            key = "%s-%s"%(bio_number,lot_number)
            volume = float(row[volumColId].value)
            if not liquidDb.has_key(key):
                liquidDb[key] = []
                liquidDb[key].append(volume)
                liquidBarcodeDb[key] = []
                liquidBarcodeDb[key].append(barcode)
            else:
                liquidDb[key].append(volume)
                liquidBarcodeDb[key].append(barcode)
        except:
            traceback.print_exc()
            pass

    n_keep = 0
    n_drop = 0
    drop_output = open("drop.txt","w")
    keep_output = open("keep.txt","w")
    for key in liquidDb.keys():
        if solidDb.has_key(key) and solidDb[key]>=2.0:
            if max(liquidDb[key])<150:
                n_drop += len(liquidDb[key])
                for id,barcode in enumerate(liquidBarcodeDb[key]):
                    print >> drop_output,barcode,key,liquidDb[key][id],"have_solid_%f"%solidDb[key]
            else:
                for id,v in enumerate(liquidDb[key]):
                    if v<150:
                        n_drop += 1
                        print >> drop_output,liquidBarcodeDb[key][id],key,v,"have_solid_%f"%solidDb[key]
                    else:
                        n_keep += 1
                        print >> keep_output,liquidBarcodeDb[key][id],key,v, "have_solid%f"%solidDb[key]
        else:
            if max(liquidDb[key])>=150:
                for id,v in enumerate(liquidDb[key]):
                    if v>=150:
                        n_keep += 1
                        print >> keep_output,liquidBarcodeDb[key][id],key,v,"liquid>=150_noSolid"
                    else:
                        n_drop += 1
                        print >> drop_output,liquidBarcodeDb[key][id],key,v,"liquid<150_noSolid"
            else:
                for id,v in enumerate(liquidDb[key]):
                    n_keep += 1
                    print >> keep_output,liquidBarcodeDb[key][id],key,v,"liquid<150_unique"


    drop_output.close()
    keep_output.close()
    print "No. liquid samples to keep:",n_keep
    print "No. liquid samples to drop:",n_drop

Usage:/Users/jfeng1/.oechem_env/lib/python2.7/site-packages/ipykernel/__main__.py input.xls
First row contains data tag names


SystemExit: 0

To exit: use 'exit', 'quit', or Ctrl-D.
